In [16]:
####画像をPDFに変換####
import numpy as np
import pandas as pd
import glob
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from PIL import Image

In [17]:
#データの読み込み
company_info = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv", index_col=0)
industry = np.array(pd.unique(company_info["業種"]))

In [18]:
#ディレクトリ名から業種コードを取得
filelist = glob.glob("D:/Statistics/data/stock_price/individual/*.csv")
industry_no = np.array(np.zeros((len(filelist), 3)), dtype="U32")

for j in range(len(filelist)):
    tmp = np.array(pd.read_csv(filelist[j].replace("\\", "/"), 
                               engine='python', encoding="utf-8")[["industry_code", "industry_name"]].iloc[0:1]).reshape(-1)
    if tmp[0] < 10:
        no = "0" + str(tmp[0])
    else:
        no = str(tmp[0])
    industry_no[j, ] = np.array([no, tmp[1], no + "_" + tmp[1]])

In [19]:
#作業フォルダ内のディレクトリだけを抽出する
base_path = r"D:/Statistics/data/image/"
image_path = glob.glob("D:/Statistics/data/image/*")
M = industry.shape[0]

In [21]:
##業界ごとに株価推移の画像をpdf化
for i in range(M):
    #pdfの設定
    target = industry_no[i, 1]
    target_name = industry_no[i, 2]
    output_path = "D:/Statistics/data/stock_price/individual/" + target_name + "/" + target_name + "の株価推移.pdf"
    pdf = canvas.Canvas(output_path, pagesize=A4)
    pdf.setPageSize((1000, 1000))

    #pdf化する画像を選択
    df = pd.DataFrame(image_path)[0]
    index = np.array(df.str.contains(industry_no[i, 1]))
    target_image = np.array(df[index])
    n = target_image.shape[0]

    #複数の画像を1つのpdfにまとめる
    for j in range(n):
        pdf.drawInlineImage(Image.open(target_image[j]), 0, 0)
        pdf.showPage()
    pdf.save()